# Fitting Type Ia Supernovae Light Curves and plotting a Hubble Diagram

In this notebook, you will learn how to fit a SNIa light curve, observed by the LCO 0.4m network as part of the Kilonova Seekers - LCO: STAR program. In theory, this should work for any SN Ia observed by LCO, as long as you reduce the data through the STAR pipeline notebook. To fit lightcurve data well using the models described here, you will ideally need at least 5 epochs of data (5 different observation times) in at least 2 filters (ideally g and r bands). 

If you have any issues, please ask for help on the STAR slack!

This is notebook inspired by the hubble constant tutorials by David Jones. 

In [ ]:
# Import dependencies 
import sncosmo                                       # cosmological libraries
import pandas as pd                                  # pandas dataframe for table manipulation
import numpy as np                                   # numpy for many maths functions
from astropy.table import Table                      # astropy tables used by sncosmo
import matplotlib.pyplot as plt                      # plotting 
from matplotlib.backends.backend_pdf import PdfPages # save plots to multi-page pdf if many filters
from sfdmap2 import sfdmap                           # to import our dust maps

In [ ]:
# Change this depending on the SN details (can find on the TNS) and the name of the photometry file. 
SN_name = "SN2025ass"
SN_redshift = 0.03
SN_ra = 161.2575525                   # in degrees
SN_dec = 76.6867034                   # in degrees
filename = "PSF_phot_1740059148.txt"  # output file from the pipeline code

In [ ]:
# Read photometry data into a pandas dataframe format to make it easier to work with
df = pd.read_table(f"../outputs/{filename}")

# Restrict to just the data we need for fitting
df = df[["filter", "mjd", "ZP", "PSFmag", "err"]]

# sncosmo works in flux, not magnitudes, so need to convert
df["flux"] = 10 ** (-0.4 * (df["PSFmag"] - df["ZP"]))
df["flux_err"] = np.abs(df["flux"] * 0.4 * np.log(10) * df["err"])

# We need to define the zero-point system for the photometric data
df["zpsys"] = "ab"

# LCO uses sdss-like filters, so we can tell sncosmo to use information from the SDSS filters
df["filter"] = "sdss" + df["filter"]
# sort the data by date
df = df.sort_values("mjd")

# sncosmo uses astropy table format, so can now convert from pandas to astropy
data = Table.from_pandas(df)

In [ ]:
# download the dust maps
!wget https://github.com/kbarbary/sfddata/archive/master.tar.gz
!tar xzf master.tar.gz

In [ ]:
# We should include dust in the Milky Way in our model as this will impact the SN parameters - this is built in to sncosmo
# Dust will make SNe appear redder and dimmer than they actually are, so it is important to take this into account
dust = sncosmo.CCM89Dust()                # using the Cardelli, Clayton, and Mathis (1989) dust dependent extinction model
dustmap = sfdmap.SFDMap("sfddata-master") # path to the Schlegel, Finkbeiner and Davis (1998) dust map downloaded earlier

# Get the amount of dust in the Milky Way along the line of sight to the SN location 
ebv = dustmap.ebv(SN_ra, SN_dec)
print(ebv)

In [ ]:
# Tell sncosmo we want to use the salt2 model for SN Ia light curves with Milky Way dust
# We can also get it to predict the amount of dust at the SN location
model = sncosmo.Model(source='salt2', effects=[dust],
                                      effect_names=['mw'], effect_frames=['obs'])

# Set the Milky Way ebv value generated above
model.set(mwebv=ebv)

# We know the redshift of this SN, so we can set this as a fixed parameter for the model to use 
model.set(z=SN_redshift)

In [ ]:
# We can now fit the model to the photometry 
result, fitted_model = sncosmo.fit_lc(data, model,
                                      ['t0', 'x0', 'x1', 'c'])

To fit SNIa lightcurves, salt2 uses the parameters t0, x0, x1 and c. 
- t0 is the date of peak magnitude measured in mjd
- x0 is a measure of the amplitude
- x1 is the shape or stretch of the light curve - think of this as how narrow or wide the lightcurve is
- c is the colour of the light curve - this can be thought of as the difference in magnitude between the filters

In [ ]:
# Plot the model fits to the photometry
fitplots = sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)

In [ ]:
# Now save the plots of the fits to a pdf
pp = PdfPages('{}-salt2fit.pdf'.format(SN_name))
sncosmo.plot_lc(data, model=fitted_model, errors=result.errors, fname=pp, format='pdf')
pp.close()

In [ ]:
# We now need to store the output parameters from the model as variables
SN_z = result.parameters[0]
SN_z_err = 0
SN_t0 = result.parameters[1]
SN_t0_err = result.errors['t0']
SN_x0 = result.parameters[2]
SN_x0_err = result.errors['x0']
SN_x1 = result.parameters[3]
SN_x1_err = result.errors['x1']
SN_c = result.parameters[4]
SN_c_err = result.errors['c']
SN_peak_mag = fitted_model.source_peakmag('bessellb', 'ab')
SN_peak_mag_err = 2.5 / np.log(10) * result.errors['x0'] / result.parameters[2]

We can now compare this SNe Ia to a larger sample of SNe Ia, used for cosmology. For this, we will use the Foundation sample. The Foundation sample contains data for 180 SNe Ia with a redshift < 0.1, all observed by the same Pan-STARRS system. You can read about it here: https://arxiv.org/abs/1711.02474  

In [ ]:
# Download data from the Foundation supernova sample
!wget https://raw.githubusercontent.com/djones1040/Foundation_DR1/refs/heads/master/Foundation_DR1.FITRES.TEXT

In [ ]:
# Read in the foundation sample data
foundation = pd.read_csv('Foundation_DR1.FITRES.TEXT', comment='#', skipinitialspace=True, sep=' ')

# Cut to just data for this analysis
foundation = foundation[['CID', 'zHD', 'zHDERR', 'mB', 'mBERR', 'x0', 'x0ERR', 'x1', 'x1ERR', 'c', 'cERR']] 

# Have a look at the data 
foundation

How does our supernova compare to the distribution of SNe Ia from the Foundation sample?  

To investigate this, we will plot the Foundation sample in blue, and indicate how our supernova compares with a dotted orange line.

In [ ]:
# Comparing distributions of the foundation sample to our single object
fig, axs = plt.subplots(1, 3, sharey=True, tight_layout=True)
fig.set_figwidth(9)
n_bins = 10
axs[0].hist(foundation['mB'], bins=n_bins, color='steelblue')
axs[0].axvline(SN_peak_mag, c='darkorange', ls=':')
axs[0].set_xlabel('mB')
axs[0].set_xlim([12,20])
axs[0].set_ylabel('# SNe Ia')
axs[1].hist(foundation['x1'], bins=n_bins, color='steelblue')
axs[1].axvline(SN_x1, c='darkorange', ls=':')
axs[1].set_xlabel('x1')
axs[1].set_xlim([-3.5,3.5])
axs[2].hist(foundation['c'], bins=n_bins, color='steelblue')
axs[2].axvline(SN_c, c='darkorange', ls=':')
axs[2].set_xlabel('c')
axs[2].set_xlim([-0.35,0.35])

plt.show()

Questions to consider:
- Is our supernova brighter or dimmer at peak compared to the sample average?
- How does the stretch (x1) compare? More positive values of x1 mean that the light-curves are wider and decline more slowly, whilst more negative values mean that the light-curves are narrower and decline faster.
- How does the colour compare? Is it redder (more positive c) or bluer (more negative c) than the sample average?

Next, let's make an uncorrected Hubble diagram of redshift versus peak brightness. We will plot our supernova in a different colour and marker style so that we can see how it compares to the Foundation sample.

In [ ]:
# Uncorrected Hubble diagram
plt.scatter(foundation['zHD'],foundation['mB'], c='steelblue', marker='.')
plt.plot(SN_z, SN_peak_mag, marker='*', c='darkorange')
plt.xlabel('redshift')
plt.ylabel('mB')
plt.show()

You may have heard that SNe Ia are "standard candles" as they all explode with the same peak brightness (dependent on their distance away from us). However, as you can see, this plot has considerable scatter! For this reason, SN Ia astronomers call them "standardisable candles" instead - by making corrections (standardisations) based on the stretch (x1) and colour (c) of the light curve, we can make them appear more similar and make this curve tighter.  

We do this by calculating the distance modulus of the supernovae, $\mu$.

We can calculate this by combining x0, x1 and c (the \"color\") using the Tripp formula (Tripp 1999).
    
$\mu = -2.5*{\rm log_{10}}(x0) + 10.635 + (\alpha\times x_1) - (\beta\times c) - \mathcal{M}$

- $\alpha$ and $\beta$ are correction terms based on each large SN sample. $\alpha$ is usually ~0.14, and $\beta$ ~ 3.1. We will use fixed values here published by the Foundation sample for their data here.
- $\mathcal{M}$ is a combination of the typical absolute magnitude of a SN Ia and the value of the Hubble constant.  If we knew H$_0$ exactly, it would be the absolute magnitude of a SN Ia in the $B$ band (a \"blue\" filter).  We'll set it to -19.36 from convention - it's just a constant.
- The 10.635 is a constant that reflects the meaning of \"amplitude\" x0 in the SALT model we have used to fit the data.
   

In [ ]:
# Correction terms for the Foundation sample
alpha =  0.143
beta =  3.218

In [ ]:
# Calculate mu for our SN
SN_mu = -2.5*np.log10(SN_x0) + 10.635 + alpha*SN_x1 - beta*SN_c + 19.36
print(f"mu = {SN_mu:.3f} mag")

In [ ]:
# Uncertainty on mu using error propagation for our SN
SN_muerr = np.sqrt((2.5/np.log(10)*SN_x0_err/SN_x0)**2.+ 
                alpha**2. * SN_x1_err**2. + 
                beta**2. * SN_c_err**2.)
print(f"mu_err = {SN_muerr:.3f} mag")

In [ ]:
# We can now do the same for the entire Foundation sample
foundation['mu'] = -2.5*np.log10(foundation['x0']) + 10.635 + alpha*foundation['x1'] - beta*foundation['c'] + 19.36
foundation['muerr'] = np.sqrt((2.5/np.log(10)*foundation['x0ERR']/foundation['x0'])**2.+ 
                alpha**2. * foundation['x1ERR']**2. + 
                beta**2. * foundation['cERR']**2.)

We can now plot a corrected version of a hubble diagram to see if the scatter has improved. We can also calculate the distance to each supernova based on what cosmological models (such as the FlatLambdaCDM model we will import) predict they should be at that redshift. We will call this the "cosmological distance". By comparing this cosmological distance, to the distance actually measured by analysis of the supernovae, we can calculate something called the "Hubble residual". 

In [ ]:
# Import a FlatLambdaCDM model
# H0=70, cosmic matter = 0.3
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(70,0.3)

In [ ]:
# Calculate predicted cosmological distance to our SN
SN_mu_mod = cosmo.distmod(SN_z).value

print(f"cosmological distance is {SN_mu_mod:.3f} mag")

In [ ]:
# Hubble residual
SN_resid = SN_mu - SN_mu_mod

print(f"Hubble residual is {SN_resid:.3f} mag")

In [ ]:
# Now we can do the same to the Foundation sample
foundation['mu_model'] = cosmo.distmod(foundation['zHD']).value
foundation['residuals'] = foundation['mu']-foundation['mu_model']

In [ ]:
# Plotting 
fig, axs = plt.subplots(2, 1,gridspec_kw={'height_ratios':[3,1]})
fig.set_figwidth(10)
plt.rcParams['figure.figsize'] = (6,8)

# Set up a curve to show the model distance vs redshift and the uncertainty on this
z = np.linspace(1e-9,0.1,100)
zerr = 250/3e5*5.0/np.log(10)*(1.0+z)/(z*(1.0+z/2.0))
axs[0].plot(z,cosmo.distmod(z).value, c = 'green')
axs[0].plot(z,cosmo.distmod(z).value+zerr, '--' , c = 'purple')
axs[0].plot(z,cosmo.distmod(z).value-zerr, '--', c = 'purple')
# add foundation to hubble plot 
axs[0].errorbar(foundation['zHD'],foundation['mu'],yerr=foundation['muerr'],fmt = '.', c='steelblue', label='Foundation Sample')
axs[0].errorbar(SN_z, SN_mu, yerr=SN_muerr, fmt='*', c='darkorange', label='{}'.format(SN_name))

# Add curve to residual plot -> curve = 0 residual
axs[1].axhline(0, c = 'green')
axs[1].plot(z,zerr, '--' , c = 'purple')
axs[1].plot(z,-zerr, '--', c = 'purple')

# Add foundation to residual plot
axs[1].errorbar(foundation['zHD'],foundation['residuals'],yerr=foundation['muerr'],fmt = '.', c='steelblue')

# Add our SN to residual plot
axs[1].errorbar(SN_z, SN_resid, yerr=SN_muerr, fmt='*', c='darkorange')

# Tidy plots and add labels
axs[0].set_xlim([0.001,0.1])
axs[1].set_xlim([0.001,0.1])
axs[0].set_ylim([30,40])
axs[1].set_ylim([-1,1])
axs[0].set_xlabel('Redshift')
axs[0].set_ylabel('Distance (Mag)')
axs[1].set_xlabel('Redshift')
axs[1].set_ylabel('Hubble Residual')
axs[0].legend(loc='upper left')

# Save figure and show 
plt.savefig('HubbleDiagram.png')
plt.show()

You have now plotted a corrected Hubble Diagram, similar to those you may have seen in cosmological analyses. The top plot is a plot of redshift versus distance, where the green line is that predicted by a cosmological LambdaCDM model and the blue points are the corrected distance measurements from the supernovae. The purple dashed line is a representation of a standard uncertainty on the model. As you can see, the model and the supernova data trace each other pretty well, with some scatter. We can gain a greater understanding of the scatter from the lower plot, known as a residual plot, which shows the differences between the model and the supernovae more clearly. This is why the green line is now at the 0 mark, as the model subtracted from itself is simply 0! 

How does our supernova compare? Is it close to or far away from the Foundation data? What does this mean? 